In [88]:
# save files
fullproc_r = '/Users/jnaiman/LMM_Figure_Parsing/resources/'

# get title and x/y labels from most frequent words
data_dir = '/Users/jnaiman/LMM_Figure_Parsing/resources/data/'

# save
fake_figs_dir = '/Users/jnaiman/Downloads/tmp/fake_figs/'

# only used to re-do inlines
dirs_latex = ['arxiv/9901', 'arxiv_new/2301', 'arxiv_new/2302']
base_dir_arxiv = '/Users/jnaiman/Dropbox/wwt_image_extraction/'

In [92]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2 as cv
import pickle
import pandas as pd
#import string
from glob import glob
from mpl_toolkits.axes_grid1 import make_axes_locatable
from PIL import ImageColor
import json

#import spacy

# this is hacktacular from: https://github.com/ReadingTimeMachine/TexSoup
# and needs to be updated
# tex_lib = '/Users/jnaiman/TexSoup/'
# from sys import path
# path.append(tex_lib)
# import TexSoup
# from TexSoup import preprocessing, postprocess

#from nltk.corpus import stopwords
import time

# from matplotlib import font_manager
# plt.rcParams['text.usetex'] = True
import matplotlib as mpl
mpl.rcParams['text.usetex'] = True
mpl.rcParams['text.latex.preamble'] = r'\usepackage{amsmath} \usepackage{amssymb}' #for \text command

#nlp = spacy.load("en_core_web_sm")

# written libs
from importlib import reload # debug
import synthetic_fig_utils # debug
reload(synthetic_fig_utils) # debug
from synthetic_fig_utils import subset_by_percent, \
 get_nrows_and_ncols, normalize_params_prob, get_ticks, get_titles_or_labels, \
 get_font_info
 #get_data, make_plot
#get_line_styles, \

import plot_parameters


# import distribution_utils
# reload(distribution_utils)
# from distribution_utils import get_random_data, get_random, \
#    get_linear, get_linear_data, get_data

import plot_utils
reload(plot_utils)
from plot_utils import get_contour_plot, get_histogram_plot, \
   get_line_plot, get_scatter_plot, make_plot

import pixel_location_utils
reload(pixel_location_utils)
from pixel_location_utils import get_errorbar_pixels, get_data_pixel_locations

import text_utils
reload(text_utils)
from text_utils import get_popular_nouns, get_inline_math

# create a bunch of fake figures
reload(synthetic_fig_utils)
from synthetic_fig_utils import normalize_params_prob
reload(plot_parameters)
from plot_parameters import plot_types_params, panel_params, \
  title_params, xlabel_params, \
  ylabel_params, aspect_fig_params, dpi_params, tight_layout_params, \
  fontsizes, base

import data_utils
reload(data_utils)
from data_utils import get_data, NumpyEncoder

use_uniques = True # use unique inlines
verbose = True

In [36]:
# get fonts -- see "cnn_create_synthetic_ticks" in FullProcess
dfont = pd.read_csv(fullproc_r + 'fonts.csv')
font_names = dfont['font name'].values

# for plot styles
plot_styles = plt.style.available

In [37]:
# get words
popular_nouns = get_popular_nouns(data_dir)

16253 343492
shape of output= (5097, 5)
shape of output= (36332, 5)


Get inline math:

In [40]:
inlines = get_inline_math(base_dir_arxiv, dirs_latex, fullproc_r,
                          recreate_inlines=False,
                         use_uniques=use_uniques)

number of inlines =  33201


## Update plotting params

In [52]:
plot_params = plot_types_params.copy()

### Line plots, Linear distributions

In [94]:
plot_params_line = {'line':plot_params['line']}

plot_params_line['line']['npoints'] = {'min':10, 'max':100}
plot_params_line['line']['nlines'] = {'min':1, 'max':10}

# just linear
plot_params_line['line']['distribution']['random']['prob'] = 0
plot_params_line['line']['distribution']['gmm']['prob'] = 0
plot_params_line['line']['distribution']['linear']['prob'] = 1

# noise
plot_params_line['line']['distribution']['linear']['noise'] = (0.25, 0.25)

plot_params_line, panel_params, \
  title_params, xlabel_params, \
  ylabel_params = normalize_params_prob(plot_params_line.copy(), panel_params, 
                                        title_params, xlabel_params, 
                                        ylabel_params)

plot_params_line

title_params capatilize did not add to 1! total = 1.2999999999999998
renormalizing...
now:  {'none': 0.07692307692307694, 'first': 0.3846153846153847, 'all': 0.5384615384615385}
xlabel_params capatilize did not add to 1! total = 1.2999999999999998
renormalizing...
now:  {'none': 0.07692307692307694, 'first': 0.3846153846153847, 'all': 0.5384615384615385}
ylabel_params capatilize did not add to 1! total = 1.2999999999999998
renormalizing...
now:  {'none': 0.07692307692307694, 'first': 0.3846153846153847, 'all': 0.5384615384615385}


{'line': {'prob': 1,
  'npoints': {'min': 10, 'max': 100},
  'line thick': {'min': 1, 'max': 5},
  'nlines': {'min': 1, 'max': 10},
  'xmin': -10000,
  'xmax': 10000,
  'ymin': -10000,
  'ymax': 10000,
  'prob same x': 0.1,
  'markers': {'prob': 0.5, 'size': {'min': 1, 'max': 20}},
  'error bars': {'elinewidth': {'min': 1, 'max': 5},
   'x': {'prob': 0.25, 'size': {'min': 0.01, 'max': 0.1}},
   'y': {'prob': 0.25, 'size': {'min': 0.01, 'max': 0.1}}},
  'distribution': {'random': {'prob': 0},
   'linear': {'prob': 1,
    'intersect': (-100, 100),
    'slope': (-5, 5),
    'noise': (0.25, 0.25)},
   'gmm': {'prob': 0,
    'histogram as line': {'prob': 0.25, 'factor': 1000},
    'nclusters': {'min': 1, 'max': 20},
    'nsamples': {'min': 10, 'max': 500},
    'cluster std': {'min': -2, 'max': 0.5},
    'noise': {'min': 0.05, 'max': 0.25}}}}}

In [95]:
xlabel_params

{'prob': 0.9,
 'n words': {'min': 1, 'max': 3},
 'capitalize': {'none': 0.07692307692307694,
  'first': 0.3846153846153847,
  'all': 0.5384615384615385},
 'equation': {'prob': 0.25}}

In [97]:
plot_params_here = plot_params_line.copy()

i = 0

datas_all = {} # for multiples
plt.close('all')

if True:
    ######### pick things ########
    # figure
    plt.set_cmap(np.random.choice(plt.colormaps()))
    
    npanels, panel_style, nrows, ncols = get_nrows_and_ncols(panel_params)
    ####npanels, panel_style,nrows,ncols = 1, 'horizontal', 1,1 # HACK
    plot_style = np.random.choice(plot_styles)
    
    aspect_fig = np.random.uniform(low=aspect_fig_params['min'], 
                                   high=aspect_fig_params['max'])
    # rows and cols 
    aspect_fig *= nrows/ncols
    dpi = int(np.random.uniform(low=dpi_params['min'], high=dpi_params['max']))
    # tight layout or not
    print('panel style:', panel_style, 'nrows,ncols=', (nrows,ncols), 
          'aspect=', aspect_fig, 'dpi=', dpi)
    
    tight_layout = True


    # get all font stuffs
    title_fontsize, xlabel_fontsize, ylabel_fontsize, \
       xlabel_ticks_fontsize, ylabel_ticks_fontsize, \
                           csfont = get_font_info(fontsizes, font_names)
    #print("ALL IS WELL HERE 1")

    # plot types?
    success = False
    start_time = time.time()
    while not success:
        with plt.style.context(plot_style):
            if tight_layout:
                fig,axes = plt.subplots(nrows,ncols,figsize=(base*nrows, base*aspect_fig*ncols), 
                                        dpi=dpi,layout='tight')
            else:
                fig,axes = plt.subplots(nrows,ncols,figsize=(base*nrows, base*aspect_fig*ncols), 
                                        dpi=dpi)
                
            if npanels == 1:
                axes = [axes]
            else: # flatten, for now
                axes = axes.flatten()

            **HERE** have to save index
                    
            ######### Generate plot data #########
            #print('here 1')
            
            choices = []; probs = []
            for k,v in plot_params_here.items():
                choices.append(k)
                probs.append(v['prob'])
        
            data_for_plots = []
            plot_types = []
            data_from_plots = []
            titles = []; xlabels = []; ylabels = []; cbars = []
            end_time = time.time()
            #try:
            if True:
                for iplot, ax in enumerate(axes):
                    start_time = time.time()
                    plot_type = np.random.choice(choices, p=probs)
                    print('PLOT TYPE:', plot_type)
                    # get distribution type
                    dist_params = plot_params_here[plot_type]['distribution'] 
                    choices_d = []; probs_d = []
                    for k,v in dist_params.items():
                        choices_d.append(k)
                        probs_d.append(v['prob'])

                    distribution_type = np.random.choice(choices_d, p=probs_d)
                    print('  Distribution Type:', distribution_type)
                    
                    #print('here 2')
                    data_for_plot = get_data(plot_params_here[plot_type],
                                    plot_type=plot_type,
                                            distribution=distribution_type)
                    end_time = time.time()
                
                    ######### PLOT ############
                    start_time = time.time()
                    #print('here 3')
                    data_from_plot, ax = make_plot(plot_params_here[plot_type], data_for_plot, 
                                         ax, plot_type=plot_type)#, plot_style=plot_style)
                    #import sys; sys.exit()
                    end_time = time.time()
                    #print('here 4')
    
                    # set ticksizes
                    ax.tick_params(axis='x', which='major', labelsize=xlabel_ticks_fontsize, labelfontfamily=csfont['fontname'])
                    ax.tick_params(axis='y', which='major', labelsize=ylabel_ticks_fontsize, labelfontfamily=csfont['fontname'])
    
                    start_time = time.time()
                    p = np.random.uniform(0,1)
                    if p < title_params['prob']:
                        title_words = get_titles_or_labels(popular_nouns, title_params['capitalize'],
                                                 title_params['equation'], inlines,
                                                 nwords=np.random.randint(low=title_params['n words']['min'],
                                                                          high=title_params['n words']['max']+1))
                        title = ax.set_title(title_words, fontsize = title_fontsize, **csfont)
                    else:
                        title = ''
                        
                    xlabel_words = get_titles_or_labels(popular_nouns, xlabel_params['capitalize'],
                                                 xlabel_params['equation'], inlines,
                                                 nwords=np.random.randint(low=xlabel_params['n words']['min'],
                                                                          high=xlabel_params['n words']['max']+1))
                    xlabel = ax.set_xlabel(xlabel_words, fontsize=xlabel_fontsize, **csfont)
                    ylabel_words = get_titles_or_labels(popular_nouns, ylabel_params['capitalize'],
                                                 ylabel_params['equation'], inlines,
                                                 nwords=np.random.randint(low=ylabel_params['n words']['min'],
                                                                          high=ylabel_params['n words']['max']+1))
                    ylabel = ax.set_ylabel(ylabel_words, fontsize=ylabel_fontsize, **csfont)
                    end_time = time.time()
            
                    # save
                    data_for_plots.append(data_for_plot)
                    plot_types.append(plot_type)
                    data_from_plots.append(data_from_plot)
                    titles.append(title)
                    xlabels.append(xlabel)
                    ylabels.append(ylabel)
    
                    if plot_type == 'scatter': # or plot_type == 'contour':
                        if 'color bar' in data_from_plot:
                            side = data_from_plot['color bar params']['side']
                            if side == 'top' or side == 'bottom':
                                orientation = 'horizontal'
                            else:
                                orientation = 'vertical'
    
                            cbar = fig.colorbar(data_from_plot['data'], 
                                         cax=data_from_plot['color bar'], 
                                         orientation=orientation)
                            cbars.append(cbar)

                    if plot_type == 'contour':
                        if 'color bar' in data_from_plot:
                            side = data_from_plot['color bar params']['side']
                            if side == 'top' or side == 'bottom':
                                orientation = 'horizontal'
                            else:
                                orientation = 'vertical'
                    
                            if 'image' in data_from_plot['data']: # select correct colorbar to use
                                datac = data_from_plot['data']['image']
                            else:
                                datac = data_from_plot['data']['contour']
                                            
                            cbar = fig.colorbar(datac, 
                                         cax=data_from_plot['color bar'], 
                                         orientation=orientation)
                            cbars.append(cbar)

    
                #if not save_smalls:
                fig.savefig(fake_figs_dir + 'Picture' + str(i+1) + '.png', dpi=dpi)
                print('saved:', fake_figs_dir + 'Picture' + str(i+1) + '.png')
                # else:
                #     fig.savefig(save_small_dir + 'Picture' + str(i+1) + '.png', dpi=dpi)
                #     print('saved:', save_small_dir + 'Picture' + str(i+1) + '.png')
                    
                success = True
            #except Exception as e:
            else:
                print('[ERROR]:')
                plt.close(fig)
                print('issue with plotting, trying again')
                print(e)
                if 'unknown color specifier' in str(e):
                    print(e)
                    import sys; sys.exit()
                if 'At least one value in the dash list must be positive' in str(e):
                    print(e)
                    print(data_from_plots)
    
    
    ####### end of plotting ############
    # try the whole thing again
    width, height = fig.canvas.get_width_height()
    # save data
    datas = {}
    # figure datas
    datas['figure'] = {'dpi':dpi, 'base':base, 'aspect ratio': aspect_fig, 
                          'nrows':nrows, 'ncols':ncols, 
                         'plot style':plot_style, 
                         'title fontsize':title_fontsize, 
                         'xlabel fontsize':xlabel_fontsize,
                         'ylabel fontsize':ylabel_fontsize}
    
    # now, get data things
    for iplot, ax in enumerate(axes):
        ###### get data from plot ######
        data_from_plot = data_from_plots[iplot]
        data_for_plot = data_for_plots[iplot]
        plot_type = plot_types[iplot]
        title = titles[iplot]
        xlabel = xlabels[iplot]
        ylabel = ylabels[iplot]
    
        data_pixels = get_data_pixel_locations(data_from_plot, plot_type, ax, width, height)
    
        # bounding box of square
        bbox = ax.get_position() # Bbox(x0, y0, x1, y1)
        xpix1 = np.array([bbox.x0,bbox.x1])
        ypix1 = np.array([bbox.y0,bbox.y1])
        xpix1 *= width
        ypix1 *= height
        
        # x-tick locations
        xticks = get_ticks(ax.get_xticklabels(), ax.get_xticklines())
    
        # y-tick locations
        yticks = get_ticks(ax.get_yticklabels(), ax.get_yticklines())
        
        # for colorbars
        colorbar_ticks = []
        if 'color bar' in data_from_plots[iplot]:
            colorbar = data_from_plots[iplot]['color bar']
            if data_from_plots[iplot]['color bar params']['side'] == 'left' \
               or data_from_plots[iplot]['color bar params']['side'] == 'right':
                ticks = colorbar.get_yticklabels()
                tick_locs = colorbar.get_yticklines(minor=False)
            else:
                ticks = colorbar.get_xticklabels()
                tick_locs = colorbar.get_xticklines(minor=False)
            colorbar_ticks = get_ticks(ticks, tick_locs)

        # title
        # Get the bounding box of the title in display space
        if title != '':
            title_bbox = title.get_window_extent()
            title_words = title.get_text()
        else:
            title_bbox = -1
            title_words = ''
    
        # xlabel
        xlabel_bbox = xlabel.get_window_extent()
        xlabel_words = xlabel.get_text()
        # ylabel
        ylabel_bbox = ylabel.get_window_extent()
        ylabel_words = ylabel.get_text()
    
    
        ########## save data ##############

        # line plot 
        plot_name = 'plot' + str(iplot) 
        datas[plot_name] = {}
        # line plot type
        datas[plot_name]['type'] = plot_type # tag for kind of plot
        datas[plot_name]['distribution'] = distribution_type
        datas[plot_name]['data'] = data_for_plot
        datas[plot_name]['data pixels'] = data_pixels
        datas[plot_name]['data from plot'] = json.loads(json.dumps(data_from_plot, cls=NumpyEncoder))
        if (plot_type == 'scatter' or plot_type == 'contour') and 'color bar' in data_from_plots[iplot]:
            #print('yes indeed')
            w = data_from_plots[iplot]['color bar'].get_window_extent()
            datas[plot_name]['color bar'] = {'xmin':w.x0,'ymin':w.y0,
                                             'xmax':w.x1,'ymax':w.y1, 
                                             'params':data_from_plot['color bar params']}
        xtmp = []
        for xt in xticks:
            l = {'data':xt[0], 'xmin': xt[1], 
                 'ymin': xt[2], 
                 'xmax':xt[3], 'ymax':xt[4],
                 'tx':xt[5], 'ty':xt[6]}
            xtmp.append(l)
        datas[plot_name]['xticks'] = xtmp.copy()
        # 
        xtmp = []
        for xt in yticks:
            l = {'data':xt[0], 'xmin': xt[1], 
                 'ymin': xt[2], 
                 'xmax':xt[3], 'ymax':xt[4], 
                'tx':xt[5], 'ty':xt[6]}
            xtmp.append(l)
        datas[plot_name]['yticks'] = xtmp.copy()
        if len(colorbar_ticks) > 0:
            xtmp = []
            for xt in colorbar_ticks:
                l = {'data':xt[0], 'xmin': xt[1], 
                     'ymin': xt[2], 
                     'xmax':xt[3], 'ymax':xt[4], 
                    'tx':xt[5], 'ty':xt[6]}
                xtmp.append(l)
            datas[plot_name]['color bar ticks'] = xtmp.copy()
            
        # axis box
        datas[plot_name]['square'] = {'xmin':xpix1[0], 'ymin':ypix1[0], 
                                         'xmax':xpix1[1], 'ymax':ypix1[1]}
        # title
        if title_bbox != -1:
            datas[plot_name]['title'] = {'xmin':title_bbox.x0, 'ymin':title_bbox.y0, 
                                            'xmax':title_bbox.x1, 'ymax':title_bbox.y1,
                                           'words':title_words}
        else:
            pass
        datas[plot_name]['xlabel'] = {'xmin':xlabel_bbox.x0, 'ymin':xlabel_bbox.y0, 
                                        'xmax':xlabel_bbox.x1, 'ymax':xlabel_bbox.y1,
                                       'words':xlabel_words}
        datas[plot_name]['ylabel'] = {'xmin':ylabel_bbox.x0, 'ymin':ylabel_bbox.y0, 
                                        'xmax':ylabel_bbox.x1, 'ymax':ylabel_bbox.y1,
                                       'words':ylabel_words}

    # # save this for small datas
    # if save_smalls:
    #     dumped = json.dumps(datas, cls=NumpyEncoder)
    #     file_path = save_small_dir + 'Picture' + str(i+1) + '.json'
    #     with open(file_path, 'w') as f:
    #         json.dump(dumped, f)
            
    # for full thing
    datas_all[i] = datas.copy()
    plt.close(fig)
print('DONE!')

selected npanels: 9
panel style: squarish nrows,ncols= (3, 3) aspect= 0.9926956075074614 dpi= 477
PLOT TYPE: line
  Distribution Type: linear
PLOT TYPE: line
  Distribution Type: linear
PLOT TYPE: line
  Distribution Type: linear
PLOT TYPE: line
  Distribution Type: linear
PLOT TYPE: line
  Distribution Type: linear
PLOT TYPE: line
  Distribution Type: linear
PLOT TYPE: line
  Distribution Type: linear
PLOT TYPE: line
  Distribution Type: linear
PLOT TYPE: line
  Distribution Type: linear
saved: /Users/jnaiman/Downloads/tmp/fake_figs/Picture1.png
DONE!


<Figure size 640x480 with 0 Axes>

In [109]:
fig

In [111]:
axes.flatten?

Docstring:
a.flatten(order='C')

Return a copy of the array collapsed into one dimension.

Parameters
----------
order : {'C', 'F', 'A', 'K'}, optional
    'C' means to flatten in row-major (C-style) order.
    'F' means to flatten in column-major (Fortran-
    style) order. 'A' means to flatten in column-major
    order if `a` is Fortran *contiguous* in memory,
    row-major order otherwise. 'K' means to flatten
    `a` in the order the elements occur in memory.
    The default is 'C'.

Returns
-------
y : ndarray
    A copy of the input array, flattened to one dimension.

See Also
--------
ravel : Return a flattened array.
flat : A 1-D flat iterator over the array.

Examples
--------
>>> a = np.array([[1,2], [3,4]])
>>> a.flatten()
array([1, 2, 3, 4])
>>> a.flatten('F')
array([1, 3, 2, 4])
Type:      builtin_function_or_method

### Generate QA Pairs from saved data

In [108]:
for k,data in datas_all.items():
    q1 = 'How many panels are in this figure?'
    q1 += ' You are a helpful assistant, please format the output as a json as {"nrows":"", "ncols":""} to store the number of rows and columns.'
    a1 = {"nrows":data['figure']['nrows'], "ncols":data['figure']['ncols']}
    print('QUESTION:', q1)
    print('ANSWER:', a1)
    print('')

QUESTION: How many panels are in this figure? You are a helpful assistant, please format the output as a json as {"nrows":"", "ncols":""} to store the number of rows and columns.
ANSWER: {'nrows': 3, 'ncols': 3}


In [110]:
data

{'figure': {'dpi': 477,
  'base': 5,
  'aspect ratio': 0.9926956075074614,
  'nrows': 3,
  'ncols': 3,
  'plot style': 'grayscale',
  'title fontsize': 20,
  'xlabel fontsize': 19,
  'ylabel fontsize': 19},
 'plot0': {'type': 'line',
  'distribution': 'linear',
  'data': {'xs': [array([-9123.19120241, -8888.77541496, -8467.10328081, -8038.23719297,
           -7705.08522168, -7208.31762126, -6845.18952034, -6800.22045706,
           -6201.16878097, -6178.64646481, -5209.35759384, -4984.00582579,
           -4857.27190429, -4399.19445146, -4388.1340696 , -4185.2129174 ,
           -2831.7639478 , -2797.71826646, -2612.1427842 , -2469.62610045,
           -2310.53707944, -1639.80865354, -1421.98621336,  -951.82284619,
            -270.31875811,  -265.36541826,     9.95686799,   733.50755187,
            1240.43361337,  1560.87032777,  1623.76951775,  1800.10812778,
            1825.41672805,  2184.34286502,  2336.60350954,  2899.20855955,
            3162.470587  ,  4023.00065219,  4445.